In [ ]:
import sys

# Adicionar path do projeto
sys.path.append('/app')

from app.services.prediction_service import ThermalPredictionService
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar visualizações
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Inicializar Serviço de Predição

In [ ]:
# Criar serviço
prediction_service = ThermalPredictionService()

print(f"MLflow URI: {prediction_service.mlflow_uri}")
print(f"Diretório de modelos: {prediction_service.model_dir}")

## 2. Carregar e Explorar Dados

In [ ]:
# Carregar dados
data_path = "/app/data/sample_thermal_data.csv"
df = pd.read_csv(data_path)

print(f"Total de registros: {len(df):,}")
print(f"\nColunas: {list(df.columns)}")
print(f"\nPeríodo: {df['timestamp'].min()} até {df['timestamp'].max()}")

df.head()

In [ ]:
# Estatísticas descritivas
df.describe()

In [ ]:
# Distribuição da sensação térmica
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.hist(df['thermal_sensation'], bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Sensação Térmica (°C)')
plt.ylabel('Frequência')
plt.title('Distribuição da Sensação Térmica')
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
comfort_counts = df['comfort_zone'].value_counts()
plt.bar(comfort_counts.index, comfort_counts.values)
plt.xlabel('Zona de Conforto')
plt.ylabel('Frequência')
plt.title('Distribuição das Zonas de Conforto')
plt.xticks(rotation=45)
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Treinar Modelos

In [ ]:
# Treinar todos os modelos
results = prediction_service.train_models(data_path)

print("\n" + "="*60)
print("RESUMO DOS RESULTADOS")
print("="*60)

for model_name, metrics in results.items():
    print(f"\n{model_name.upper()}:")
    print(f"  Test RMSE: {metrics['test_rmse']:.4f}°C")
    print(f"  Test MAE:  {metrics['test_mae']:.4f}°C")
    print(f"  Test R²:   {metrics['test_r2']:.4f}")

## 4. Comparar Modelos

In [ ]:
# Comparação visual
model_names = list(results.keys())
test_rmse = [results[m]['test_rmse'] for m in model_names]
test_mae = [results[m]['test_mae'] for m in model_names]
test_r2 = [results[m]['test_r2'] for m in model_names]

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# RMSE
axes[0].bar(model_names, test_rmse, color='steelblue')
axes[0].set_ylabel('RMSE (°C)')
axes[0].set_title('Root Mean Squared Error')
axes[0].set_ylim([0, max(test_rmse) * 1.2])
axes[0].grid(alpha=0.3)

# MAE
axes[1].bar(model_names, test_mae, color='coral')
axes[1].set_ylabel('MAE (°C)')
axes[1].set_title('Mean Absolute Error')
axes[1].set_ylim([0, max(test_mae) * 1.2])
axes[1].grid(alpha=0.3)

# R²
axes[2].bar(model_names, test_r2, color='lightgreen')
axes[2].set_ylabel('R²')
axes[2].set_title('R² Score')
axes[2].set_ylim([0, 1])
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Testar Predições

In [ ]:
# Teste com dados de exemplo
test_cases = [
    {
        'name': 'Dia Quente de Verão',
        'temperature': 32.0,
        'humidity': 70.0,
        'wind_velocity': 3.5,
        'pressure': 1013.0,
        'solar_radiation': 850.0
    },
    {
        'name': 'Noite Fria de Inverno',
        'temperature': 12.0,
        'humidity': 80.0,
        'wind_velocity': 15.0,
        'pressure': 1020.0,
        'solar_radiation': 0.0
    },
    {
        'name': 'Tarde Confortável',
        'temperature': 24.0,
        'humidity': 60.0,
        'wind_velocity': 5.0,
        'pressure': 1015.0,
        'solar_radiation': 400.0
    }
]

for case in test_cases:
    print(f"\n{'='*60}")
    print(f"CASO: {case['name']}")
    print(f"{'='*60}")
    
    # Predição com Random Forest
    pred_rf = prediction_service.predict(
        temperature=case['temperature'],
        humidity=case['humidity'],
        wind_velocity=case['wind_velocity'],
        pressure=case['pressure'],
        solar_radiation=case['solar_radiation'],
        model_name='random_forest'
    )
    
    print("\nEntradas:")
    print(f"  Temperatura: {case['temperature']}°C")
    print(f"  Umidade: {case['humidity']}%")
    print(f"  Vento: {case['wind_velocity']} km/h")
    print(f"  Pressão: {case['pressure']} hPa")
    print(f"  Radiação Solar: {case['solar_radiation']} W/m²")
    
    print("\nResultados:")
    print(f"  Sensação Física: {pred_rf['physical_sensation']}°C ({pred_rf['physical_comfort_zone']})")
    if 'ml_prediction' in pred_rf:
        print(f"  Predição ML: {pred_rf['ml_prediction']}°C ({pred_rf['ml_comfort_zone']})")
        print(f"  Diferença: {pred_rf['prediction_difference']}°C")

## 6. Análise de Feature Importance

In [ ]:
# Feature importance do Random Forest
if 'random_forest' in prediction_service.models:
    model = prediction_service.models['random_forest']
    
    # Nomes das features
    feature_names = [
        'temperature', 'humidity', 'wind_velocity', 'pressure', 'solar_radiation',
        'hour_sin', 'hour_cos', 'day_sin', 'day_cos',
        'temp_humidity_interaction', 'wind_chill_factor', 'radiation_normalized', 'pressure_deviation'
    ]
    
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    plt.figure(figsize=(12, 6))
    plt.bar(range(len(importances)), importances[indices])
    plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=45, ha='right')
    plt.xlabel('Features')
    plt.ylabel('Importância')
    plt.title('Feature Importance - Random Forest')
    plt.tight_layout()
    plt.show()
    
    print("\nTop 5 Features mais importantes:")
    for i in range(min(5, len(indices))):
        print(f"{i+1}. {feature_names[indices[i]]}: {importances[indices[i]]:.4f}")

## 7. Conclusões

**Modelos treinados com sucesso!**

Os modelos estão salvos em `/app/models` e registrados no MLflow.

**Para usar os modelos via API:**
```bash
# Predição única
curl -X POST "http://localhost:8060/prediction/predict" \
  -H "Content-Type: application/json" \
  -d '{
    "temperature": 28.0,
    "humidity": 65.0,
    "wind_velocity": 5.0,
    "pressure": 1013.0,
    "solar_radiation": 600.0
  }'
```

**Acessar MLflow:**
- URL: http://localhost:5000
- Visualizar experimentos, métricas e artefatos